In [114]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import os
from sklearn.metrics import f1_score

In [115]:
os.chdir('/home/baek/Downloads/')
x_train = pd.read_csv('x_train_oct.csv')

In [116]:
y_train = pd.read_csv('y_train_oct.csv')

In [120]:
from sklearn.utils import resample
df = pd.concat([x_train, y_train], axis=1)
class_counts=df.iloc[:,-1].value_counts()

min_class = class_counts.idxmin()
min_count = class_counts.min()

balanced_data = pd.DataFrame()
for class_index in class_counts.index:
    class_subset = df[df.iloc[:,-1] == class_index]
    resampled_subset = resample(class_subset,
                                replace=False,
                                n_samples=min_count,
                                random_state=123)  # Ensure reproducibility
    balanced_data = pd.concat([balanced_data, resampled_subset], axis=0)
    
X_balanced = balanced_data.iloc[:,:-1]
y_balanced = balanced_data.iloc[:,-1]

In [130]:
clf_3 = RandomForestClassifier(oob_score = True)
clf_3.fit(x_train, y_train['0'])

RandomForestClassifier(oob_score=True)

## Run this code below: and add to doc

In [ ]:
clf_3_cross_val = cross_val_score(clf_3, x_train, y_train['0'], cv=4)

In [133]:
print(f"The oob score for the full model with no class weights: {clf_3.oob_score_}")

The oob score for the full model with no class weights: 0.8026098464253106


In [131]:
clf_4 = RandomForestClassifier(oob_score = True)
clf_4.fit(X_balanced, y_balanced)

RandomForestClassifier(oob_score=True)

In [141]:
clf_4_cross_val = cross_val_score(clf_4, x_train, y_train['0'], cv=4)

In [142]:
print(clf_4_cross_val)

[0.79208043 0.79748861 0.79260536 0.79519061]


In [136]:
print(f"the oob score for the balanced model with no class weights: {clf_4.oob_score_}")

the oob score for the balanced model with no class weights: 0.6845176683002321


In [134]:
clf_5 = RandomForestClassifier(oob_score = True , class_weight= {0:7, 1:7, 2:2.5, 3:1}, n_estimators = 200)
clf_5.fit(x_train, y_train['0'])

RandomForestClassifier(class_weight={0: 7, 1: 7, 2: 2.5, 3: 1},
                       n_estimators=200, oob_score=True)

In [143]:
clf_5_cross_val = cross_val_score(clf_5, x_train, y_train['0'], cv=4)

In [144]:
print(clf_5_cross_val)

[0.79363972 0.80003283 0.79457507 0.79748861]


In [137]:
print(f"The oob score for the full model with class weights and increased estimators: {clf_5.oob_score_}")

The oob score for the full model with class weights and increased estimators: 0.8079752146660237


In [138]:
clf_6 = RandomForestClassifier(oob_score = True, class_weight= {0:7, 1:7, 2:2.5, 3:1}, n_estimators = 200)
clf_6.fit(X_balanced, y_balanced)

RandomForestClassifier(class_weight={0: 7, 1: 7, 2: 2.5, 3: 1},
                       n_estimators=200, oob_score=True)

## Run this code below and add to doc

In [145]:
clf_6_cross_val = cross_val_score(clf_6, x_train, y_train['0'], cv=4)

KeyboardInterrupt: 

In [139]:
print(f"The oob score for the balanced model with class weights and increased estimators: {clf_6.oob_score_}")

The oob score for the balanced model with class weights and increased estimators: 0.704023729687903


In [121]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters grid
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [None, 10, 20],      # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4]     # Minimum samples required at each leaf node
}

# Create the random forest classifier
rf_clf = RandomForestClassifier()

# Perform grid search with 5-fold cross-validation
grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=5, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_balanced, y_balanced)

# Print the best hyperparameters found
print("Best hyperparameters:", grid_search.best_params_)

Best hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
